In [1]:
import math, itertools, random
import altair as alt

# Lab 6-b: Buscando a oriC 🐠🐠

## 1. Patrones superpuestos
En el último lab vimos algunos conceptos de probabilidad y combinatoria que nos van a ser útiles para analizar secuencias en Python. En concreto, vimos cómo calcular la probabilidad de ver un k-mero en una secuencia de `N` caracteres del alfabeto genómico. 

Sin embargo, en entregas también anteriores vimos que los k-meros pueden superponerse, lo cual es un problema: por ejemplo, la función `count()` deja de funcionar apropiadamente. Vamos a ver algún problema relacionado más con este tipo patrones y por qué es importante conocerlos. 

Para ilustrar el problema de los patrones superpuestos, vamos a jugar a un juego llamado "La mejor apuesta".
En este juego, el jugador 1 elige un k-mero A (por simplicidad usaremos el alfabeto binario `{0, 1}`, aunque funciona igual para otros alfabetos más largos) y el jugador 2, sabiendo qué ha elegido el jugador 1, selecciona otro k-mero B.


En el juego, se irá componiendo una secuencia de 0s y 1s de la siguiente manera: en cada ronda, se tira una moneda al aire y si sale cara, se añade un 1. Si sale cruz, se añade un 0. Se gana cuando el k-mero elegido por el jugador aparece antes que el del contrincante.

In [ ]:
a = "111010"
b = "000101"

In [ ]:
from IPython.display import clear_output
from time import sleep
from random import randint

sec = ''
while(True):
    for c in ['—', '\\', '|', '/', '—'] * 2:
        print(c)
        print('')
        print('')
        sleep(0.3)
        clear_output(wait=True)
    
    resultado = random.randint(0, 1)
    cara_cruz = "cara"
    if resultado == 0:
        cara_cruz = "cruz"

    print('— %s!! (Añado %d)' % (cara_cruz, resultado))
    sec+=str(resultado)
    print('Secuencia: %s' % sec)
    if a in sec:
        print("Fin! El jugador 1 gana!")
        break
    if b in sec:
        print("Fin! El jugador 2 gana!")
        break
    sleep(2.5)
              

## 1.1 Muchas partidas
Vaya, ¡¿qué pasa?! ¿por qué suele ganar el jugador 2? ¿Habrá sido suerte?

Como esto va un poco lento, vamos a acelerarlo creando una función que simula n partidas.

In [ ]:
def simular(kmer_a, kmer_b, n=100000, retardo_secs=0):
    print("Simulando %d juegos" % n)
    ganadores = []
    kmer_a_str = "".join([str(a) for a in kmer_a])
    kmer_b_str = "".join([str(a) for a in kmer_b])
    print("K-mero %s (A) contra %s (B)" % (kmer_a_str, kmer_b_str))
    for i in range(n):
#         print('Ronda %d/%d' % (i+1, n))
        resultado = ""
        while True:
            resultado += str(random.randint(0, 1))
            if kmer_a_str in resultado:
                ganadores.append("A")
                break
            if kmer_b_str in resultado:
                ganadores.append("B")
                break
                
#         if ganadores.count("A") == 0 or ganadores.count("B") == 0:
#           continue

#         print("El jugador 1 ganó %d veces, mientras que el 2 ganó %d veces (dev=%f%%)" % (ganadores.count("A"), 
#                                                    ganadores.count("B"), 
#                                                    abs(1-ganadores.count("B")/ganadores.count("A"))))
#         clear_output(wait=True)
#         sleep(retardo_secs)
            
    return ganadores

In [ ]:
a = "111010"
b = "100101"
ganadores = simular(a, b, n=100000, retardo_secs=0.0)
print("El jugador 1 ganó %d veces, mientras que el 2 ganó %d veces (dev=%f%%)" % (ganadores.count("A"), 
                                                       ganadores.count("B"), 
                                                       abs(1-ganadores.count("B")/ganadores.count("A"))))

## 1.2 ¿Por qué ocurre esto?

¡Resulta que no todos los k-meros tienen la misma probabilidad de salir!
¿Por qué? Lo vas a ver tu mism@. 

### Ejercicio 1:
Genera todas las posibles variaciones de unos y ceros de longitud 5. Después, comprueba cuántas veces aparecen la "01"  y "11".

__¿Cómo explicarías los resultados?__

In [3]:
## solución ejercicio 1
todas = ["".join(a) for a in list(itertools.product("10", repeat=5))]
print(todas)
cerouno = "01"
unouno = "11"
veces_01 = 0
veces_11 = 0

for a in todas:
    if cerouno in a:
        veces_01+=1
    if unouno in a:
        veces_11+=1
        
print("%s aparece %d/%d veces (prob %.2f%%) y %s aparece %d/%d veces (prob %.2f%%)" % (cerouno, 
                                                                             veces_01, 
                                                                             len(todas),
                                                                             100*veces_01/len(todas),
                                                                             unouno,
                                                                             veces_11,
                                                                             len(todas),
                                                                             100*veces_11/len(todas)))

['11111', '11110', '11101', '11100', '11011', '11010', '11001', '11000', '10111', '10110', '10101', '10100', '10011', '10010', '10001', '10000', '01111', '01110', '01101', '01100', '01011', '01010', '01001', '01000', '00111', '00110', '00101', '00100', '00011', '00010', '00001', '00000']
01 aparece 26/32 veces (prob 81.25%) y 11 aparece 19/32 veces (prob 59.38%)


#### __Conclusión__

Las probabilidades de encontrar un k-mero en una secuencia dependen de si este permite ocurrencias solapadas o no. 
¡Cuidado con esto!


## 2. La replicación es un proceso asimétrico

Vamos a seguir tratando de buscar el origen de replicación en genomas procarióticos usando métodos computacionales. 

Resulta que, debido a ciertos procesos que tienen lugar durante la replicación (como la __deaminación__), vamos a encontrar discrepancias entre las cantidades medidas y esperadas de guanina y citosina dependiendo de la zona del genoma en la que nos encontremos (ver figura).

![skew](img/increasing_decreasing_skew.png)

Vamos a utilizar esta información para tratar de ubicar el origen de replicación en un genoma circular de una bacteria. 

__Pregunta:__ ¿Cómo hallarías el origen de replicación de acuerdo a la figura?

Vamos a corroborar nuestra hipótesis con el genoma de la ya conocida Thermotoga Petrophila. 

In [2]:
def lee_genoma(ruta_fichero):
  fd = open(ruta_fichero)
  genome = fd.read()
  fd.close()
  return genome
  

In [3]:
tp_genoma = lee_genoma('../data/Thermotoga-petrophila.txt')
ec_genoma = lee_genoma('../data/E-coli.txt')
vc_genoma = lee_genoma('../data/vibrio_cholerae.txt')

In [4]:
len(tp_genoma)

1823511

### Ejercicio 2:
Como sabemos que C tiene que ser más frecuente en una mitad de la cadena que en otro, vamos a deslizar una ventana gigante de tamaño `len(genoma)/2` para medir cómo la citosina va cambiando según recorremos la secuencia. De todas las ventanas que consideremos, aquella con la menor frecuencia absoluta de citosinas será la mitad ori-ter en sentido 5'->3', y la que tenga más, la mitad ter-ori. 

Crea una función `lista_bases(genoma, base)` que recoja la variación de la frecuencia absoluta de una base que se pasa como argumento en una ventana deslizante de tamaño `len(genoma)/2` (irá contando el número de veces que aparece la base en cada ventana). Pruéba tu código pasando el genoma de la Thermotoga Petrophila y 'C'(citosina).

__¡Cuidado! ¡Ten en cuenta que el genoma es circular!__


In [5]:
def lista_base(genoma, base):
    lista_freqs = []
    n = len(genoma)
    genoma_extendido = genoma + genoma[0:n//2]
    for i in range(len(genoma)):
        lista_freqs.append(genoma_extendido[i:i+(n//2)].count("C"))
    return lista_freqs

In [6]:
len(lista_base(tp_genoma, 'C')) # Tarda mucho!

KeyboardInterrupt: 

### Acelerando el algoritmo

Probablemente, el código que has hecho tarde mucho. Vamos a intentar acelerarlo. 
Tal como hemos pensado el algoritmo, tenemos que hacer `len(genoma)` búsquedas de 'C' en `len(genoma)//2` elementos. 

1. ¿Cuántos elementos ha de recorrer nuestro algoritmo para terminar, sabiendo que la longitud del genoma es 1.823.511 bp? 
2. ¿Cómo podemos arreglarlo? 

Implementa un método `lista_base_rapida(genoma, base)` que haga lo mismo pero más rápido. 

Para arreglar nuestro algoritmo, evitaremos buscar repetidamente la base en cada una de las ventanas. 
Para ello consideraremos sólo el elemento que se "pierde" a la izquierda y el que se "gana" a la derecha cada vez que deslizamos la ventana: 
1. Si el elemento que se pierde es del tipo de la base que buscamos, entonces la siguiente ventana tendrá seguro una base menos. 
2. Si el elemento que se gana es del tipo de la base que buscamos, entonces la siguiente ventana tendrá seguro una base más.

In [7]:
def lista_base_rapida(genoma, base):
    lista_freqs = []
    n = len(genoma)
    genoma_extendido = genoma + genoma[0:n//2]    
    
    lista_freqs.append(genoma[:n//2].count(base))
    for i in range(1, n):
        lista_freqs.append(lista_freqs[i-1])
        
        if genoma_extendido[i-1] == base:
            lista_freqs[i] = lista_freqs[i]-1
        if genoma_extendido[i+(n//2)-1] == base:
            lista_freqs[i] = lista_freqs[i]+1
            
    return lista_freqs

Lo que devuelva esta función no va a ser fácil de interpretar si no lo visualizamos. 

¿Solución? ¡Altair al poder! Vamos a declarar una función que genera un diagrama de línea con los valores y posiciones que le pasemos, así: 

In [8]:
def diagrama_linea(data, agg=1000, zero=True):
    """
    Dibuja un diagrama de líneas con la lista que se le pase en data de la forma [0, 1, 0, 5, ...]
    agg: Tamaño de la agregación 
    zero: Si se trunca el eje y o no. 
    """
    data_aggregated = []
    i = 0
    for j in range(agg, len(data), agg):
        data_aggregated.append(sum(data[i:j]))
        i = j

    return alt.Chart(alt.Data(
        values=[{"i": i*agg, "v": v/agg} for i,v in enumerate(data_aggregated)])
             ).mark_line().encode(
        x="i:Q",
        y=alt.Y("v:Q", scale=alt.Scale(zero=zero)))

Ahora sí, vamos a ver qué ocurre con las citosinas en el genoma de la Thermotoga Petrophila. 

__¿Dónde diremos que va a estar ubicado el origen de replicación?__

In [9]:
diagrama_linea(lista_base_rapida(tp_genoma, 'C'), zero=False)

alt.Chart(...)

In [10]:
diagrama_linea(lista_base_rapida(ec_genoma, 'C'), zero=False)

alt.Chart(...)

### Ejercicio 3: 

Aunque la visualización nos da una muy buena idea de lo que está ocurriendo, a ojímetro va a estar un poco complicado sacar la posición exacta. 

Implementa una función `min_vals_pos(lista_base)` que devuelva las posiciones donde la lista de variación de la base generada con `lista_base_rapida` que se pasa como argumento alcanza un valor mínimo. 

In [11]:
def min_vals_pos(lista_base):
    positions = []
    global_min = min(lista_base)
    for i,item in enumerate(lista_base):
        if item == global_min:
            positions.append(i)
    return positions

In [12]:
min_vals_pos(lista_base_rapida(tp_genoma, 'C'))

[898983, 898996, 898997]

In [13]:
min_vals_pos(lista_base_rapida(ec_genoma, 'C'))

[3971149]

### Diagramas de sesgo (skew diagrams)

Aunque la variación de citosinas en el genoma nos puede dar una buena idea de por dónde se encuentra el origen de replicación, lo cierto es que existe otra manera más precisa de encontrarlo: el diagrama de sesgo.

El diagrama de sesgo se construye teniendo en cuenta la diferencia en Cs y Gs en las dos medias secuencias (sentido y antisentido). En la media secuencia en sentido, habrá un decremento en citosinas (#G - #C será positivo), mientras que en el antisentido se percibirá un decremento en guaninas (#G - #C será negativo).

Por lo tanto, como hacíamos en el anterior caso podemos analizar en qué posiciones cambia esta tendencia en el genoma para localizar _ori_.

![deamina](img/deamina.png)


El diagrama de sesgo se construye de manera análoga al anterior caso, pero ahora simplemente tendremos que ir anotando el número de Gs y Cs que se vayan encontrando al recorrer el genoma de manera que:

1. El sesgo inicial (posición 0) será 0.
2. Sumaremos 1 al sesgo calculado para la anterior posición cada vez que encontremos una guanina.
3. Restaremos 1 al sesgo calculado para la anterior posición cada vez que encontremos una citosina.

Por ej. el skew de la secuencia `CATGGGCATCGGCCATACGCC` es:
![skew_diagram](img/skew_diagram.png)

### Ejercicio 4:
Implementa una función `lista_sesgo(secuencia)` que devuelva una lista con el sesgo G/C para cada posición del genoma que se pasa como argumento.
Prueba tu código con la secuencia de la Thermotoga Petrophila y visualízala usando las funciones proporcionadas.

In [ ]:
def lista_sesgo(secuencia):
    sesgo = [0]
    for i in range(0, len(secuencia)):
        if secuencia[i] == 'C':
            sesgo.append(sesgo[i] - 1)
        elif secuencia[i] == 'G':
            sesgo.append(sesgo[i] + 1)
        else:
            sesgo.append(sesgo[i])
    return sesgo
        

## OriC en E.Coli

Usando el diagrama de sesgo podemos identificar fácilmente el origen de replicación en E.Coli. ¿Dónde está?
Compáralo con los resultados obtenidos usando el método simple de contar la variación de citosinas. 

Pruébalo con otros genomas. ¿Qué ocurre?

In [ ]:
diagrama_linea(lista_sesgo(ec_genoma)) & diagrama_linea(lista_base_rapida(ec_genoma, 'C'), zero=False)

In [ ]:
min_skew(ec_genome)

### Extra: Explicación patrones superpuestos

El juego de la mejor apuesta para k-meros se basa en la paradoja de las palabras superpuestas que hemos visto al inicio de este notebook. Este juego es muy especial (y complicado!) ya que es un __juego no transitivo__, es decir, si A gana a B, y B gana C, no necesariamente ocurre que A gana a C. 

Además, resulta que para k>2, el jugador B (el que elige el k-mero en segundo lugar) siempre puede elegir un k-mero que tenga más posibilidades de ganar. Esto se basa en el concepto de i-superposición y polinomio de correlación:

Diremos que B está i-superpuesto con A si los i últimos dígitos de A coinciden con los i últimos dígitos de B. 
Por ejemplo, como se muestra en la figura, "110110" se 1-,2- y 5-superpone a "011011".

![i-overlap](img/i-overlap.png)

Basándonos en esta definición, podemos definir el concepto de correlación entre dos k-meros A y B que es una palabra de k letras binarias (0 ó 1) tal que $c_{i} = 1 \iff$ B se (k-i)-superpone con A, o $c_{i} = 0 \iff$ si no. Esta correlación se puede capturar con el polinomio de correlación:

$K_{A,B}(t) = c_{0} + c_{1} \cdot t + c_{2} \cdot t^{2} + \dots + c_{k-1} \cdot t^{k-1}$


El matemático John Conway, experto en teoría de juegos dio una simple ecuación para calcular la probabilidad de que el k-mero B derrote al k-mero A:

$\frac{K_{A,A} - K_{A,B}}{K_{B,B} - K_{B,A}}$

Conway nunca publicó una demostración de esta fórmula (aunque sí otros después). En aquel entonces, Martin Gardner, un conocido escritor en matemáticas dijo: _"No tengo ni idea de por qué funciona. Simplemente saca la solución como por arte de magia, como otros muchos de los algoritmos de Conway."_

In [ ]:
def get_correlation(kmer_a, kmer_b):
    assert(isinstance(kmer_a, type("tuple")) == isinstance(kmer_b, type("tuple")))
    assert(len(kmer_a) == len(kmer_b))
    
    corr = [int(kmer_a[i:] == kmer_b[:(len(kmer_b) - i)]) for i in range(len(kmer_a))]
    
    return tuple(corr)

a = (0, 1, 1, 0, 1, 1)
b = (1, 1, 0, 1, 1, 0)

get_correlation(a, b)

In [ ]:
def get_corr_poly(kmer_a, kmer_b, p=1/2):
    corr = get_correlation(kmer_a, kmer_b)
    poly = [float(v) * math.pow(p, i) for i,v in enumerate(corr)]
    return poly
    

In [ ]:
def get_best_odds(kmer_a, p=1/2):
    probs_map = {}
    
    kaa = sum(get_corr_poly(kmer_a, kmer_a, p))
    for kmer in itertools.product([0, 1], repeat=len(kmer_a)):
        if kmer == kmer_a:
            continue
        probs_map[kmer] = (kaa - sum(get_corr_poly(kmer_a, kmer, p))) / (sum(get_corr_poly(kmer, kmer, p)) - sum(get_corr_poly(kmer, kmer_a, p)))
    
    m = max(probs_map.values())
    print("max prob: %f" % m)
    best_bets = []
    for kmer in probs_map:
        if kmer == kmer_a:
            continue
        if probs_map[kmer] == m:
            print("adding %s to best bets" % str(kmer))
            best_bets.append(kmer)
    
    print("Best bets against %s: %s" % (kmer_a, str(best_bets)))
    return best_bets

In [ ]:
import random
def simulate(kmer_a, kmer_b, n=100000):
    print("Simulating %d plays" % n)
    winners = []
    kmer_a_str = "".join([str(a) for a in kmer_a])
    kmer_b_str = "".join([str(a) for a in kmer_b])
    print("Will play %s (A) against %s (B)" % (kmer_a_str, kmer_b_str))
    for i in range(n):
        res_string = ""
        while True:
            res_string += str(random.randint(0, 1))
            if kmer_a_str in res_string:
                winners.append("A")
                break
            if kmer_b_str in res_string:
                winners.append("B")
                break
    return winners

a = (1, 0, 0, 1)
b = get_best_odds(a, p=1/2)[0] # Computa el k-mero con la mayor probabilidad de ganar. 

winners = simulate(a, b, n=10000)
print("A won %d times, while B won %d times (%f%%)" % (winners.count("A"), 
                                                       winners.count("B"), 
                                                       winners.count("B")/winners.count("A")))
        
        
    